<a href="https://colab.research.google.com/github/divyadharshini29/CSK-Detector/blob/main/CSK_Experiment_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Training Detectron2 Model
import torch
assert torch.__version__.startswith("1.8")
import torchvision
import cv2
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
def get_data_dicts(directory, classes):
dataset_dicts = []
for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
json_file = os.path.join(directory, filename)
with open(json_file) as f:
img_anns = json.load(f)
record = {}
filename = os.path.join(directory, img_anns["imagePath"])
record["file_name"] = filename
record["height"] = 500
record["width"] = 900
annos = img_anns["shapes"]
objs = []
for anno in annos:
px = [a[0] for a in anno['points']] # x coord
py = [a[1] for a in anno['points']] # y-coord
poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
poly = [p for x in poly for p in x]
obj = {
"bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
"bbox_mode": BoxMode.XYXY_ABS,
"segmentation": [poly],
"category_id": classes.index(anno['label']),
"iscrowd": 0
}
58
objs.append(obj)
record["annotations"] = objs
dataset_dicts.append(record)
return dataset_dicts
classes = ['countertop', 'dining_sofa', 'bottle', 'cabinates', 'tea_pot', 'cookies', 'door', 'ceining_lamp', 'faucet',
'bathtub', 'bath_sink', 'mirror', 'dirty_plates', 'oven_mitten', 'pot', 'crockery', 'cooking_range',
'knives', 'paper_towel', 'forks', 'food', 'washer_dryer', 'storage_box', 'folded_cloth', 'shelf',
'laundry_basket', 'person', 'veggie', 'cutting_board', 'pan', 'spatula', 'salad', 'ipad', 'coffee_maker',
'coffee_press', 'oven', 'tv', 'bookcase', 'carpet', 'sink', 'clothes', 'toilet', 'bath_towel', 'hand_soap',
'toilet_paper', 'dresse', 'table_lamp', 'hood', 'bed', 'small_table', 'cushion', 'night_stand', 'armchair',
'closet', 'shower', 'table', 'laptop', 'cup', 'stairs', 'water_heater', 'lamp', 'dog', 'book',
'laundry_detergent', 'window', 'AC', 'indoor_plant', 'toy', 'guitar', 'coffee_table', 'vase', 'picture',
'couch', 'chair', 'plate', 'rug', 'pillow', 'pencils', 'dining_table', 'glass', 'clock', 'fruits', 'books']
data_path = '/content/drive/MyDrive/Upwork Project/folders/'
for d in ["train", "val"]:
DatasetCatalog.register(
"category_" + d,
lambda d=d: get_data_dicts(data_path+d, classes)
)
MetadataCatalog.get("category_" + d).set(thing_classes=classes)
microcontroller_metadata = MetadataCatalog.get("category_train")
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = () # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
59
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml") # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 1000 # 300 iterations seems good enough, but you can certainly train longer
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 83 # 3 classes (data, fig, hazelnut)
# cfg.OUTPUT_DIR = '/content/gdrive/MyDrive/My_Results/'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
cfg.OUTPUT_DIR
f = open('./output/config.yml', 'w')
f.write(cfg.dump())
f.close()
!zip -r /content/drive/MyDrive/Model.zip /content/output
# Inference using model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("val", )
predictor = DefaultPredictor(cfg)
test_dataset_dicts = get_data_dicts(data_path+'val', classes)
test_dataset_dicts
for d in random.sample(test_dataset_dicts, 4):
img = cv2.imread(d["file_name"])
outputs = predictor(img)
v = Visualizer(img[:, :, ::-1],
metadata=microcontroller_metadata,
scale=0.8,
instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()
60
# saving model on drive
model_save_name = 'detectron2_house_model.pth'
path = F"/content/drive/MyDrive/detectron/{model_save_name}"
torch.save(trainer.model.state_dict(), path)
#saving on colab
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
Evaluating the model on non-annotated images
import torch
import torchvision
import cv2
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.modeling import build_model, build_roi_heads
from detectron2.config import get_cfg
61
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/detectron/Model.zip', 'r') as zip_ref:
zip_ref.extractall('/content/drive/MyDrive/detectron/output')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file("/content/drive/MyDrive/detectron/output/content/output/config.yml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/detectron/output/content/output/model_final.pth") # path to the model we just trained
predictor = DefaultPredictor(cfg)
cfg.DATASETS.TEST = ("/content/drive/MyDrive/detectron/test", )
test_metadata = MetadataCatalog.get("/content/drive/MyDrive/detectron/test")
def get_data_dicts(directory, classes):
dataset_dicts = []
for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
json_file = os.path.join(directory, filename)
with open(json_file) as f:
img_anns = json.load(f)
record = {}
filename = os.path.join(directory, img_anns["imagePath"])
record["file_name"] = filename
record["height"] = 500
record["width"] = 900
annos = img_anns["shapes"]
objs = []
for anno in annos:
px = [a[0] for a in anno['points']] # x coord
py = [a[1] for a in anno['points']] # y-coord
poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
poly = [p for x in poly for p in x]
obj = {
"bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
"bbox_mode": BoxMode.XYXY_ABS,
"segmentation": [poly],
"category_id": classes.index(anno['label']),
"iscrowd": 0
}
objs.append(obj)
62
record["annotations"] = objs
dataset_dicts.append(record)
return dataset_dicts
classes = ['countertop', 'dining_sofa', 'bottle', 'cabinates', 'tea_pot', 'cookies', 'door', 'ceining_lamp', 'faucet',
'bathtub', 'bath_sink', 'mirror', 'dirty_plates', 'oven_mitten', 'pot', 'crockery', 'cooking_range',
'knives', 'paper_towel', 'forks', 'food', 'washer_dryer', 'storage_box', 'folded_cloth', 'shelf',
'laundry_basket', 'person', 'veggie', 'cutting_board', 'pan', 'spatula', 'salad', 'ipad', 'coffee_maker',
'coffee_press', 'oven', 'tv', 'bookcase', 'carpet', 'sink', 'clothes', 'toilet', 'bath_towel', 'hand_soap',
'toilet_paper', 'dresse', 'table_lamp', 'hood', 'bed', 'small_table', 'cushion', 'night_stand', 'armchair',
'closet', 'shower', 'table', 'laptop', 'cup', 'stairs', 'water_heater', 'lamp', 'dog', 'book',
'laundry_detergent', 'window', 'AC', 'indoor_plant', 'toy', 'guitar', 'coffee_table', 'vase', 'picture',
'couch', 'chair', 'plate', 'rug', 'pillow', 'pencils', 'dining_table', 'glass', 'clock', 'fruits', 'books']
data_path = '/content/drive/MyDrive/detectron/'
for d in ["train", "val"]:
DatasetCatalog.register(
"category_" + d,
lambda d=d: get_data_dicts(data_path+d, classes)
)
MetadataCatalog.get("category_" + d).set(thing_classes=classes)
microcontroller_metadata = MetadataCatalog.get("category_train")
from detectron2.utils.visualizer import ColorMode
import glob
# for imageName in glob.glob('/content/drive/MyDrive/DSC00339.JPG'):
im = cv2.imread('/content/drive/MyDrive/detectron/test/1_kitchen resized.jpg')
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
#metadata=test_metadata,
metadata=microcontroller_metadata,
scale=1
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#imshow(out.get_image()[:, :, ::-1])
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()
#Collecting Output Instances
instances = outputs["instances"]
detected_class_indexes = instances.pred_classes
63
prediction_boxes = instances.pred_boxes
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
class_catalog = metadata.thing_classes
# Create an empty list for class names
class_name_list=[]
for idx, coordinates in enumerate(prediction_boxes):
class_index = detected_class_indexes[idx]
class_name = class_catalog[class_index]
class_name_list.append(class_name)
print(class_name_list)
#Culinary Robot
kitchen = ['dishwasher,' 'pot', 'crockery', 'cupboard', 'chair', 'table', 'tv', 'drawer', 'freezer', 'knives', 'coffee_maker', 'spatula', 'oven', 'cup', 'ladle', 'toaster', 'saucepan', 'egg', 'bread', 'butter', 'veggie', 'salad', 'muffin', 'hot_sauce', 'cook', 'plate', 'glass', 'dining_table', 'hood', 'cutting_board', 'pan', 'drink', 'fork']
def kitchen_score(class_name_list):
score_kitchen = 0
count = 0
for item in class_name_list:
if item in kitchen:
if item == 'refrigerator':
score_kitchen += 0.58
count += 1
elif item == 'glass':
score_kitchen += 0.92
count += 1
elif item == 'chair':
score_kitchen += 0.75
count += 1
elif item == 'plate':
score_kitchen += 0.92
count += 1
elif item == 'microwave':
score_kitchen = 0.75
count += 1
elif item == 'countertop':
score_kitchen = 0.58
count += 1
elif item == 'sink':
score_kitchen = 0.58
count += 1
elif item == 'cabinates':
64
score_kitchen = 0.96
count += 1
elif item == 'food':
score_kitchen = 0.58
count += 1
elif item == 'fruits':
score_kitchen = 0.46
count += 1
elif item == 'spices':
score_kitchen = 0.91
count += 1
elif item == 'rice_cooker':
score_kitchen = 0.58
count += 1
elif item == 'tea_pot':
score_kitchen = 0.46
count += 1
elif item == 'blender':
score_kitchen = 0.87
count += 1
elif item == 'dishwasher':
score_kitchen = 0.75
count += 1
elif item == 'pot':
score_kitchen = 0.96
count += 1
elif item == 'crockery':
score_kitchen = 0.46
count += 1
elif item == 'cupboard':
score_kitchen = 0.67
count += 1
elif item == 'table':
score_kitchen = 0.58
count += 1
elif item == 'tv':
score_kitchen = 0.46
count += 1
elif item == 'drawer':
score_kitchen = 0.58
count += 1
elif item == 'freezer':
score_kitchen = 0.87
count += 1
elif item == 'knives':
65
score_kitchen = 1
count += 1
elif item == 'coffee_maker':
score_kitchen = 0.46
count += 1
elif item == 'spatula':
score_kitchen = 0.58
count += 1
elif item == 'oven':
score_kitchen = 0.58
count += 1
elif item == 'cup':
score_kitchen = 0.58
count += 1
elif item == 'ladle':
score_kitchen = 0.46
count += 1
elif item == 'toaster':
score_kitchen = 0.46
count += 1
elif item == 'saucepan':
score_kitchen = 0.58
count += 1
elif item == 'egg':
score_kitchen = 0.46
count += 1
elif item == 'bread':
score_kitchen = 0.58
count += 1
elif item == 'butter':
score_kitchen = 0.58
count += 1
elif item == 'veggie':
score_kitchen = 0.46
count += 1
elif item == 'salad':
score_kitchen = 0.67
count += 1
elif item == 'muffin':
score_kitchen = 0.5
count += 1
elif item == 'hot_sauce':
score_kitchen = 0.46
count += 1
elif item == 'cook':
66
score_kitchen = 0.58
count += 1
elif item == 'dining_table':
score_kitchen = 0.92
count += 1
elif item == 'hood':
score_kitchen = 0.92
count += 1
elif item == 'cutting_board':
score_kitchen = 0.92
count += 1
elif item == 'pan':
score_kitchen = 0.46
count += 1
elif item == 'couch':
score_kitchen = 0.25
count += 1
elif item == 'drink':
score_kitchen = 0.46
count += 1
elif item == 'fork':
score_kitchen = 0.58
count += 1
return score_kitchen, count
#Laundry Robot
laundry = ['clothes','t_shirt','detergent','bleach','table','washer_dryer','wash_cloth','ironing_board','dirty_clothes','shelf','paper_towel','trash_can','towel_rack','water_heater','plumbing_pipes','drain','tile','laundry_basket','coumputer','stairs']
def laundry_score(class_name_list):
laundry_score = 0
count1 = 0
for item in class_name_list:
if item in laundry:
if item == 'clothes':
laundry_score += 0.82
count1 += 1
elif item == 't_shirt':
laundry_score += 0.58
count1 += 1
elif item == 'detergent':
laundry_score += 0.6
count1 += 1
elif item == 'bleach':
67
laundry_score += 0.58
count1 += 1
elif item == 'table':
laundry_score = 0.67
count1 += 1
elif item == 'washer_dryer':
laundry_score = 0.58
count1 += 1
elif item == 'wash_cloth':
laundry_score = 0.46
count1 += 1
elif item == 'ironing_board':
laundry_score = 0.62
count1 += 1
elif item == 'dirty_clothes':
laundry_score = 0.56
count1 += 1
elif item == 'shelf':
laundry_score = 0.46
count1 += 1
elif item == 'paper_towel':
laundry_score = 0.46
count1 += 1
elif item == 'trash_can':
laundry_score = 0.46
count1 += 1
elif item == 'towel_rack':
laundry_score = 0.67
count1 += 1
elif item == 'water_heater':
laundry_score = 0.66
count1 += 1
elif item == 'plumbing_pipes':
laundry_score = 0.92
count1 += 1
elif item == 'drain':
laundry_score = 0.58
count1 += 1
elif item == 'tile':
laundry_score = 0.67
count1 += 1
elif item == 'laundry_basket':
laundry_score = 0.81
count1 += 1
elif item == 'computer':
68
laundry_score = 0.19
count1 += 1
elif item == 'stairs':
laundry_score = 0.46
count1 += 1
return laundry_score, count1
#Vacuum Robot
vacuum = ['floor','linoleum','carpet','trash','couch','sofa','coffee_table','closet','hallway','foyer','basement','stairs'
def vacuum_score(class_name_list):
vacuum_score = 0
count2 = 0
for item in class_name_list:
if item in vacuum:
if item == 'floor':
vacuum_score += 0.58
count2 += 1
elif item == 'linoleum':
vacuum_score += 0.46
count2 += 1
elif item == 'carpet':
vacuum_score += 0.46
count2 += 1
elif item == 'trash':
vacuum_score += 0.58
count2 += 1
elif item == 'couch':
vacuum_score = 0.81
count2 += 1
elif item == 'sofa':
vacuum_score = 0.46
count2 += 1
elif item == 'coffee_table':
vacuum_score = 0.46
count2 += 1
elif item == 'closet':
vacuum_score = 0.46
count2 += 1
elif item == 'hallway':
vacuum_score = 0.58
count2 += 1
elif item == 'foyer':
69
vacuum_score = 0.58
count2 += 1
elif item == 'basement':
vacuum_score = 0.67
count2 += 1
elif item == 'stairs':
vacuum_score = 0.67
count2 += 1
return vacuum_score, count2
#Living Room
living_room = ['sofa','cushion','coffee_table','pillow','couch','sofa','rug','books','armchair']
def living_score(class_name_list):
living_score = 0
count3 = 0
for item in class_name_list:
if item in living_room:
if item == 'sofa':
living_score += 0.75
count3 += 1
elif item == 'cushion':
living_score += 0.46
count3 += 1
elif item == 'coffee_table':
living_score += 0.58
count3 += 1
elif item == 'pillow':
living_score += 0.46
count3 += 1
elif item == 'couch':
living_score = 0.81
count3 += 1
elif item == 'sofa':
living_score = 0.46
count3 += 1
elif item == 'rug':
living_score = 0.46
count3 += 1
elif item == 'books':
living_score = 0.58
count3 += 1
elif item == 'armchair':
living_score = 0.46
70
count3 += 1
return living_score, count3
#Bedroom
bedroom = ['bed', 'nightstand', 'table_lamp', 'clock', 'toy']
def bedroom_score(class_name_list):
bedroom_score = 0
count4 = 0
for item in class_name_list:
if item in bedroom:
if item == 'bed':
bedroom_score += 0.46
count4 += 1
elif item == 'nightstand':
bedroom_score += 0.46
count4 += 1
elif item == 'table_lamp':
bedroom_score += 0.92
count4 += 1
elif item == 'clock':
bedroom_score += 0.67
count4 += 1
elif item == 'toy':
bedroom_score = 0.81
count4 += 1
return vacuum_score, count4
#Bathroom
bathroom = ['toilet', ' toilet_paper', 'sink', 'bath_towel', 'shower','bath_tub', 'hand_soap', 'faucet']
def bathroom_score(class_name_list):
bathroom_score = 0
count5 = 0
for item in class_name_list:
if item in bathroom:
if item == 'toilet':
bathroom_score += 0.81
count5 += 1
elif item == 'toilet_paper':
bathroom_score += 0.46
count5 += 1
elif item == 'sink':
bathroom_score += 0.58
count5 += 1
71
elif item == 'bath_towel':
bathroom_score += 0.46
count5 += 1
elif item == 'shower':
bathroom_score = 0.58
count5 += 1
elif item == 'bath_tub':
bathroom_score = 0.58
count5 += 1
elif item == 'hand_soap':
bathroom_score = 0.67
count5 += 1
elif item == 'faucet':
bathroom_score = 0.87
count5 += 1
return bathroom_score, count5
#Results
# Cooking robot, kitchen
if kitchen_score(class_name_list)[1] == 0:
print("Cooking robot: 0%")
print("Kitchen: 0%")
else:
print("Cooking robot: ", kitchen_score(class_name_list)[0]/kitchen_score(class_name_list)[1]*100, "%")
print("Kitchen: ", kitchen_score(class_name_list)[0]/kitchen_score(class_name_list)[1]*100, "%")
#Laundry robot
if laundry_score(class_name_list)[1] == 0:
print("Laundry robot: 0%")
else:
print("Laundry robot: ", laundry_score(class_name_list)[0]/laundry_score(class_name_list)[1]*100, "%")
#Vacuum robot
if vacuum_score(class_name_list)[1] == 0:
print("Vacuum robot: 0%")
else:
print("Vacuum robot: ", laundry_score(class_name_list)[0]/laundry_score(class_name_list)[1]*100, "%")
#Living room
if living_score(class_name_list)[1] == 0:
print("Living room: 0%")
else:
print("Living room: ", living_score(class_name_list)[0]/living_score(class_name_list)[1]*100, "%")
#Bedroom
72
if bedroom_score(class_name_list)[1] == 0:
print("Bedroom: 0%")
else:
print("Bedroom: ", bedroom_score(class_name_list)[0]/bedroom_score(class_name_list)[1]*100, "%")
#Bathroom
if bathroom_score(class_name_list)[1] == 0:
print("Bathroom: 0%")
else:
print("Bathroom: ", bathroom_score(class_name_list)[0]/bathroom_score(class_name_list)[1]*100, "%")
Web-camera implementation
import torch
import torchvision
import cv2
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
73
from detectron2.modeling import build_model, build_roi_heads
from detectron2.config import get_cfg
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/detectron/Model.zip', 'r') as zip_ref:
zip_ref.extractall('/content/drive/MyDrive/detectron/output')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file("/content/drive/MyDrive/detectron/output/content/output/config.yml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/detectron/output/content/output/model_final.pth") # path to the model we just trained
predictor = DefaultPredictor(cfg)
def get_data_dicts(directory, classes):
dataset_dicts = []
for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
json_file = os.path.join(directory, filename)
with open(json_file) as f:
img_anns = json.load(f)
record = {}
filename = os.path.join(directory, img_anns["imagePath"])
record["file_name"] = filename
record["height"] = 500
record["width"] = 900
annos = img_anns["shapes"]
objs = []
for anno in annos:
px = [a[0] for a in anno['points']] # x coord
py = [a[1] for a in anno['points']] # y-coord
poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
74
poly = [p for x in poly for p in x]
obj = {
"bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
"bbox_mode": BoxMode.XYXY_ABS,
"segmentation": [poly],
"category_id": classes.index(anno['label']),
"iscrowd": 0
}
objs.append(obj)
record["annotations"] = objs
dataset_dicts.append(record)
return dataset_dicts
classes = ['countertop', 'dining_sofa', 'bottle', 'cabinates', 'tea_pot', 'cookies', 'door', 'ceining_lamp', 'faucet',
'bathtub', 'bath_sink', 'mirror', 'dirty_plates', 'oven_mitten', 'pot', 'crockery', 'cooking_range',
'knives', 'paper_towel', 'forks', 'food', 'washer_dryer', 'storage_box', 'folded_cloth', 'shelf',
'laundry_basket', 'person', 'veggie', 'cutting_board', 'pan', 'spatula', 'salad', 'ipad', 'coffee_maker',
'coffee_press', 'oven', 'tv', 'bookcase', 'carpet', 'sink', 'clothes', 'toilet', 'bath_towel', 'hand_soap',
'toilet_paper', 'dresse', 'table_lamp', 'hood', 'bed', 'small_table', 'cushion', 'night_stand', 'armchair',
'closet', 'shower', 'table', 'laptop', 'cup', 'stairs', 'water_heater', 'lamp', 'dog', 'book',
'laundry_detergent', 'window', 'AC', 'indoor_plant', 'toy', 'guitar', 'coffee_table', 'vase', 'picture',
'couch', 'chair', 'plate', 'rug', 'pillow', 'pencils', 'dining_table', 'glass', 'clock', 'fruits', 'books']
data_path = '/content/drive/MyDrive/detectron/'
for d in ["train", "val"]:
DatasetCatalog.register(
"category_" + d,
lambda d=d: get_data_dicts(data_path+d, classes)
)
MetadataCatalog.get("category_" + d).set(thing_classes=classes)
microcontroller_metadata = MetadataCatalog.get("category_train")
def js_to_image(js_reply):
"""
Params:
js_reply: JavaScript object containing image from webcam
Returns:
img: OpenCV BGR image
"""
# decode base64 image
image_bytes = b64decode(js_reply.split(',')[1])
# convert bytes to numpy array
jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
75
# decode numpy array into OpenCV BGR image
img = cv2.imdecode(jpg_as_np, flags=1)
return img
def video_stream():
js = Javascript('''
var video;
var div = null;
var stream;
var captureCanvas;
var imgElement;
var labelElement;
var pendingResolve = null;
var shutdown = false;
function removeDom() {
stream.getVideoTracks()[0].stop();
video.remove();
div.remove();
video = null;
div = null;
stream = null;
imgElement = null;
captureCanvas = null;
labelElement = null;
}
function onAnimationFrame() {
if (!shutdown) {
window.requestAnimationFrame(onAnimationFrame);
}
if (pendingResolve) {
var result = "";
if (!shutdown) {
captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
result = captureCanvas.toDataURL('image/jpeg', 0.8)
}
var lp = pendingResolve;
pendingResolve = null;
lp(result);
}
}
async function createDom() {
if (div !== null) {
return stream;
}
76
div = document.createElement('div');
div.style.border = '2px solid black';
div.style.padding = '3px';
div.style.width = '100%';
div.style.maxWidth = '600px';
document.body.appendChild(div);
const modelOut = document.createElement('div');
modelOut.innerHTML = "Status:";
labelElement = document.createElement('span');
labelElement.innerText = 'No data';
labelElement.style.fontWeight = 'bold';
modelOut.appendChild(labelElement);
div.appendChild(modelOut);
video = document.createElement('video');
video.style.display = 'block';
video.width = div.clientWidth - 6;
video.setAttribute('playsinline', '');
video.onclick = () => { shutdown = true; };
stream = await navigator.mediaDevices.getUserMedia(
{video: { facingMode: "environment"}});
div.appendChild(video);
imgElement = document.createElement('img');
imgElement.style.position = 'absolute';
imgElement.style.zIndex = 1;
imgElement.onclick = () => { shutdown = true; };
div.appendChild(imgElement);
const instruction = document.createElement('div');
instruction.innerHTML =
'' +
'When finished, click here or on the video to stop this demo';
div.appendChild(instruction);
instruction.onclick = () => { shutdown = true; };
video.srcObject = stream;
await video.play();
captureCanvas = document.createElement('canvas');
captureCanvas.width = 640; //video.videoWidth;
captureCanvas.height = 480; //video.videoHeight;
window.requestAnimationFrame(onAnimationFrame);
return stream;
}
77
async function stream_frame(label, imgData) {
if (shutdown) {
removeDom();
shutdown = false;
return '';
}
var preCreate = Date.now();
stream = await createDom();
var preShow = Date.now();
if (label != "") {
labelElement.innerHTML = label;
}
if (imgData != "") {
var videoRect = video.getClientRects()[0];
imgElement.style.top = videoRect.top + "px";
imgElement.style.left = videoRect.left + "px";
imgElement.style.width = videoRect.width + "px";
imgElement.style.height = videoRect.height + "px";
imgElement.src = imgData;
}
var preCapture = Date.now();
var result = await new Promise(function(resolve, reject) {
pendingResolve = resolve;
});
shutdown = false;
return {'create': preShow - preCreate,
'show': preCapture - preShow,
'capture': Date.now() - preCapture,
'img': result};
}
''')
display(js)
def video_frame(label):
data = eval_js('stream_frame("{}")'.format(label))
return data
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
78
while True:
js_reply = video_frame(label_html)
if not js_reply:
break
# convert JS response to OpenCV Image
img = js_to_image(js_reply["img"])
outputs = predictor(img)
v = Visualizer(img[:, :, ::-1],
#metadata=test_metadata,
metadata=microcontroller_metadata,
scale=1.2
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#imshow(out.get_image()[:, :, ::-1])
plt.figure(figsize = (14, 10))
imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()
Re-seizing images
from PIL import Image
import os, sys
path = "/home/irina/Desktop/thesis/detectron2/imgs_for_resize/"
dirs = os.listdir( path )
def resize():
for item in dirs:
if os.path.isfile(path+item):
im = Image.open(path+item)
f, e = os.path.splitext(path+item)
imResize = im.resize((900,500), Image.ANTIALIAS)
imResize.save(f + ' resized.jpg', 'JPEG', quality=90)
resize()